In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-02-16 15:42:34.130742: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 15:42:41.232524: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-16 15:42:41.232631: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-16 15:42:42.795692: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-16 15:42:45.755726: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-16 15:42:45.763548: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
batchsize = 8

#training model with data data and splitting it into 
training and validation

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range = 0.2, shear_range = 0.2, zoom_range = 0.2,width_shift_range = 0.2, height_shift_range = 0.2, validation_split = 0.2)
train_data = train_datagen.flow_from_directory(r'/home/blackson/Desktop/Projects/Machine_Learning/Driver_Drowsiness_ML_Project/PreparedData/', target_size =(80,80), batch_size = batchsize, class_mode = 'categorical', subset = 'training')
validation_data = train_datagen.flow_from_directory(r'/home/blackson/Desktop/Projects/Machine_Learning/Driver_Drowsiness_ML_Project/PreparedData/train/', target_size = (80,80), batch_size = batchsize, class_mode = 'categorical', subset = 'validation')

Found 67920 images belonging to 2 classes.
Found 16084 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_data = test_datagen.flow_from_directory(r'/home/blackson/Desktop/Projects/Machine_Learning/Driver_Drowsiness_ML_Project/PreparedData/test', target_size =(80,80), batch_size = batchsize, class_mode = 'categorical')


Found 4474 images belonging to 2 classes.


In [7]:
bmodel = InceptionV3(include_top = False, weights = 'imagenet', input_tensor = Input(shape = (80,80,3), batch_size = 8))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation = 'relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation = 'softmax')(hmodel)

model = Model(inputs = bmodel.input, outputs = hmodel)
for layer in bmodel.Layers:
    layer.trainable = False

 3465216/87910968 [>.............................] - ETA: 1:25:53

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
checkpoint = ModelCheckpoint(r'/home/blackson/Desktop/Projects/Machine_Learning/Driver_Drowsiness_ML_Project/models/', monitor = 'val_loss', save_best_only = True, verbose = 3)
earlystop = EarlyStopping(monitor = 'val_loss', patience = 7, verbose = 3, restore_best_weights = True)
learning_rate = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 3)

In [ ]:
model.compile(optimizer = Adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit_generator(train_data, steps_per_epoch = train_data.samples//batchsize, validation_data = validation_data, validation_steps = validation_data.sample//batchsize)

In [ ]:
# model evaluation

In [ ]:
acc_tr, loss_tr = model.evaluate_generator(train_data) # evaluation of accuracy and loss of train data
print(acc_tr)
print(loss_tr)

In [ ]:
acc_vr, loss_vr = model.evaluate_generator(validation_data) # evaluation of accuracy and loss of validation data
print(acc_vr)
print(loss_vr)

In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data) # evaluation of accuracy and loss of test  data
print(acc_test)
print(loss_test)